In [2]:
!pip install spacy

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 16.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 668.8/668.8 KB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 21.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 KB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.1/128.1 KB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 KB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 24.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.6/457.6 KB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 KB 25.9 MB/s eta 0:00:00
You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.


In [4]:
!python -m spacy download de_core_news_sm

/home/user/.local/lib/python3.9/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /home/user/pytorch/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 13.5 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [21]:
from processing.text_processor import preprocess_transcript_for_alignment, preprocess_transcript_for_sentence_split, split_to_sentences
from transcribing.stt_transcribe import get_stt_transcription

In [28]:
import os
import glob
import nemo.collections.asr as nemo_asr
import torch

In [30]:
torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(torch.__version__)
print(device)

1.11.0
cpu


In [10]:
PATH_TO_TRANSCRIPT = 'data/text_processed_2021-02-24.txt'

In [11]:
with open(PATH_TO_TRANSCRIPT, encoding='utf-8') as f:
    transcript = f.read()
    transcript = preprocess_transcript_for_sentence_split(transcript)
    truth_sentences = split_to_sentences(transcript)

In [ ]:
#print(truth_sentences)

In [13]:
PATH_TO_AUDIO = '/data/voice/SZ_parliament/raw/parts'

In [ ]:
audio_files = glob.glob(os.path.join(PATH_TO_AUDIO, '*.wav'))
#print(audio_files)

In [22]:
# set a single pregenerated file for testing
TEST_FILE = ['test.wav']

In [25]:
# load model once
model = nemo_asr.models.EncDecCTCModelBPE.from_pretrained(model_name="stt_de_conformer_ctc_large")

[NeMo I 2022-05-31 10:53:50 cloud:56] Found existing object /home/user/.cache/torch/NeMo/NeMo_1.8.1/stt_de_conformer_ctc_large/6ccf7bc56339a48fad7e06265e367dfa/stt_de_conformer_ctc_large.nemo.
[NeMo I 2022-05-31 10:53:50 cloud:62] Re-using file from: /home/user/.cache/torch/NeMo/NeMo_1.8.1/stt_de_conformer_ctc_large/6ccf7bc56339a48fad7e06265e367dfa/stt_de_conformer_ctc_large.nemo
[NeMo I 2022-05-31 10:53:50 common:747] Instantiating model from pre-trained checkpoint
[NeMo I 2022-05-31 10:53:53 mixins:165] Tokenizer SentencePieceTokenizer initialized with 128 tokens


[NeMo W 2022-05-31 10:53:53 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 8
    shuffle: true
    num_workers: 16
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    shuffle_n: 2048
    is_tarred: true
    tarred_audio_filepaths: /data/train/audio__OP_0..1023_CL_.tar
    
[NeMo W 2022-05-31 10:53:53 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath:
    - /data/voxpopuli_de/dev/voxpopuli_dev_manifest.json
    - /data/voxpopuli_de/test/voxpopuli_

[NeMo I 2022-05-31 10:53:53 features:259] PADDING: 0
[NeMo I 2022-05-31 10:53:53 features:276] STFT using torch
[NeMo I 2022-05-31 10:53:56 save_restore_connector:209] Model EncDecCTCModelBPE was successfully restored from /home/user/.cache/torch/NeMo/NeMo_1.8.1/stt_de_conformer_ctc_large/6ccf7bc56339a48fad7e06265e367dfa/stt_de_conformer_ctc_large.nemo.


In [26]:
stt_output = get_stt_transcription(model, TEST_FILE)

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

[NeMo W 2022-05-31 10:54:03 nemo_logging:349] /home/user/.local/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
      warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
    


In [31]:
#print(stt_output)

die zur verfügung stehende mittel veraiufteilt werde um ebene all unter namehöhne zu unterstütze wo jetzt dringend ousi hilf bruchet zerwähne ich filli als anders gefäß dass wie viele betroffenen unternahme personal höschte durch kurzarbeitsentschädigung oder ebe durch corona erwerbsersatzentschädigung und das seit märz zweitausend


In [32]:
true_transcript = "Gleichzeitig sollen die zur Verfügung stehenden Mittel fair aufgeteilt werden, um sämtliche Unternehmen unterstützen zu können, die jetzt dringend unsere Hilfe brauchen. Zu erwähnen ist auch, dass bei vielen betroffenen Unternehmen die Personalkosten durch die Kurzarbeitsentschädigung oder durch die Corona-Erwerbsersatzentschädigung seit März 2020 gedeckt"

In [50]:
print(true_transcript)

Gleichzeitig sollen die zur Verfügung stehenden Mittel fair aufgeteilt werden, um sämtliche Unternehmen unterstützen zu können, die jetzt dringend unsere Hilfe brauchen. Zu erwähnen ist auch, dass bei vielen betroffenen Unternehmen die Personalkosten durch die Kurzarbeitsentschädigung oder durch die Corona-Erwerbsersatzentschädigung seit März 2020 gedeckt


In [ ]:
!pip install biopython

In [ ]:
from aligning.sentence_aligner import SentenceAligner

In [ ]:
PATH_TO_SENTENCE_ALIGNER = 'aligner_model/sentence_aligner__i4ds_alignment_corpus__amazon_transcribe.pickle'

In [ ]:
sentence_aligner = SentenceAligner()

In [ ]:
sentence_alignment, _, _ = sentence_aligner.predict_one(truth_sentences, 
                                                        stt_output,
                                                        do_length_ratio_full_transcript_filtering=False,
                                                        do_time_correction=False
                                                        )

In [ ]:
print(stt_output)

In [42]:
from aligning.sentence_aligner import SentenceAligner

In [43]:
PATH_TO_SENTENCE_ALIGNER = 'aligner_model/sentence_aligner__i4ds_alignment_corpus__amazon_transcribe.pickle'

In [46]:
sentence_aligner = SentenceAligner()

In [51]:
sentence_alignment, _, _ = sentence_aligner.predict_one(truth_sentences, 
                                                        stt_output,
                                                        do_length_ratio_full_transcript_filtering=False,
                                                        do_time_correction=False
                                                        )

AttributeError: 'str' object has no attribute 'start_time'

In [52]:
print(stt_output)

die zur verfügung stehende mittel veraiufteilt werde um ebene all unter namehöhne zu unterstütze wo jetzt dringend ousi hilf bruchet zerwähne ich filli als anders gefäß dass wie viele betroffenen unternahme personal höschte durch kurzarbeitsentschädigung oder ebe durch corona erwerbsersatzentschädigung und das seit märz zweitausend
